In [1]:
import numpy as np
import pandas as pd

In [2]:
dataset = pd.read_csv(r'C:\Users\Sagar Kumar Sharma\MapUp-Data-Assessment-F\datasets\dataset-3.csv')
dataset.head()

,id_start,id_end,distance
0,1001400,1001402,9.7
1,1001402,1001404,20.2
2,1001404,1001406,16.0
3,1001406,1001408,21.7
4,1001408,1001410,11.1


# Question 1: Distance Matrix Calculation
Create a function named calculate_distance_matrix that takes the dataset-3.csv as input and generates a DataFrame representing distances between IDs.

The resulting DataFrame should have cumulative distances along known routes, with diagonal values set to 0. If distances between toll locations A to B and B to C are known, then the distance from A to C should be the sum of these distances. Ensure the matrix is symmetric, accounting for bidirectional distances between toll locations (i.e. A to B is equal to B to A).

In [3]:
def calculate_distance_matrix(dataset):
    #dictionary to store cumulative distances
    cumulative_distances = {}

    #iteration on the rows of dataset
    for index, row in dataset.iterrows():
        start_id = row['id_start']
        end_id = row['id_end']
        distance = row['distance']

        #cumulative distance update for both directions (a to b and b to a)
        cumulative_distances.setdefault(start_id, {}).setdefault(end_id, 0)
        cumulative_distances.setdefault(end_id, {}).setdefault(start_id, 0)

        cumulative_distances[start_id][end_id] += distance
        cumulative_distances[end_id][start_id] += distance

    #create dataframe fro m the dictionary
    distance_matrix_df = pd.DataFrame.from_dict(cumulative_distances, orient='index')
    distance_matrix_df = distance_matrix_df.fillna(0)  # Replace NaN values with 0

    return distance_matrix_df

resulting_distance_matrix = calculate_distance_matrix(dataset)

In [4]:
print(resulting_distance_matrix)

           1001402.0  1001400.0  1001404.0  1001406.0  1001408.0  1001410.0  \
1001400.0        9.7        0.0        0.0        0.0        0.0        0.0   
1001404.0       20.2        0.0        0.0       16.0        0.0        0.0   
1001402.0        0.0        9.7       20.2        0.0        0.0        0.0   
1001406.0        0.0        0.0       16.0        0.0       21.7        0.0   
1001408.0        0.0        0.0        0.0       21.7        0.0       11.1   
1001410.0        0.0        0.0        0.0        0.0       11.1        0.0   
1001412.0        0.0        0.0        0.0        0.0        0.0       15.6   
1001414.0        0.0        0.0        0.0        0.0        0.0        0.0   
1001416.0        0.0        0.0        0.0        0.0        0.0        0.0   
1001418.0        0.0        0.0        0.0        0.0        0.0        0.0   
1001420.0        0.0        0.0        0.0        0.0        0.0        0.0   
1001422.0        0.0        0.0        0.0        0.

# Question 2: Unroll Distance Matrix
Create a function unroll_distance_matrix that takes the DataFrame created in Question 1. The resulting DataFrame should have three columns: columns id_start, id_end, and distance.

All the combinations except for same id_start to id_end must be present in the rows with their distance values from the input DataFrame.

In [5]:
def unroll_distance_matrix(distance_matrix_df):
    #creat an empty Df to store unrolled distances
    unrolled_df = pd.DataFrame(columns=['id_start', 'id_end', 'distance'])
    for id_start, row in distance_matrix_df.iterrows():
        for id_end, distance in row.items():
            #here we are skipping rows where id_start is equal to id_end
            if id_start != id_end:
                unrolled_df = unrolled_df.append({
                    'id_start': id_start,
                    'id_end': id_end,
                    'distance': distance
                }, ignore_index=True)

    return unrolled_df

# Assuming resulting_distance_matrix is the DataFrame from Question 1
resulting_unrolled_df = unroll_distance_matrix(resulting_distance_matrix)

In [6]:
print(resulting_unrolled_df)

       id_start     id_end  distance
0     1001400.0  1001402.0       9.7
1     1001400.0  1001404.0       0.0
2     1001400.0  1001406.0       0.0
3     1001400.0  1001408.0       0.0
4     1001400.0  1001410.0       0.0
...         ...        ...       ...
1801  1001472.0  1001462.0       0.0
1802  1001472.0  1001464.0       0.0
1803  1001472.0  1001466.0       0.0
1804  1001472.0  1001468.0       0.0
1805  1001472.0  1001470.0      16.0

[1806 rows x 3 columns]


# Question 3: Finding IDs within Percentage Threshold
Create a function find_ids_within_ten_percentage_threshold that takes the DataFrame created in Question 2 and a reference value from the id_start column as an integer.

Calculate average distance for the reference value given as an input and return a sorted list of values from id_start column which lie within 10% (including ceiling and floor) of the reference value's average.

In [7]:
def find_ids_within_ten_percentage_threshold(unrolled_df, reference_value):
    #filter rows with the specified reference value in id_start column
    reference_rows = unrolled_df[unrolled_df['id_start'] == reference_value]

    #calculate the avg distance for the reference value
    average_distance = reference_rows['distance'].mean()

    #define the percent threshold
    percentage_threshold = 10

    #calculate the lower and upper bounds
    lower_bound = average_distance - (average_distance * percentage_threshold / 100)
    upper_bound = average_distance + (average_distance * percentage_threshold / 100)

    #filter rows within the 10% threshold
    within_threshold = unrolled_df[
        (unrolled_df['id_start'] != reference_value) &  # Exclude the reference value itself
        (unrolled_df['distance'] >= lower_bound) &
        (unrolled_df['distance'] <= upper_bound)
    ]

    result_ids = sorted(within_threshold['id_start'].unique())

    return result_ids


reference_value = 1001402
result_ids_within_threshold = find_ids_within_ten_percentage_threshold(resulting_unrolled_df, reference_value)

In [8]:
result_ids_within_threshold

[1001444.0, 1004355.0]

# Question 4: Calculate Toll Rate
Create a function calculate_toll_rate that takes the DataFrame created in Question 2 as input and calculates toll rates based on vehicle types.

The resulting DataFrame should add 5 columns to the input DataFrame: moto, car, rv, bus, and truck with their respective rate coefficients. The toll rates should be calculated by multiplying the distance with the given rate coefficients for each vehicle type:

0.8 for moto
1.2 for car
1.5 for rv
2.2 for bus
3.6 for truck

In [9]:
import pandas as pd

def calculate_toll_rate(unrolled_df):
    #define rate coefficients for each vehicle
    rate_coefficients = {'moto': 0.8, 'car': 1.2, 'rv': 1.5, 'bus': 2.2, 'truck': 3.6}

    #iterate through the rate coefficients and calculate toll rates
    for vehicle_type, rate_coefficient in rate_coefficients.items():
        #create a new column for the toll rate based on the distance and rate coefficient
        unrolled_df[vehicle_type] = unrolled_df['distance'] * rate_coefficient

    return unrolled_df


resulting_df_with_toll_rates = calculate_toll_rate(resulting_unrolled_df)

In [10]:
resulting_df_with_toll_rates

,id_start,id_end,distance,moto,car,rv,bus,truck
0,1001400.0,1001402.0,9.7,7.76,11.64,14.55,21.34,34.92
1,1001400.0,1001404.0,0.0,0.00,0.00,0.00,0.00,0.00
2,1001400.0,1001406.0,0.0,0.00,0.00,0.00,0.00,0.00
3,1001400.0,1001408.0,0.0,0.00,0.00,0.00,0.00,0.00
4,1001400.0,1001410.0,0.0,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...
1801,1001472.0,1001462.0,0.0,0.00,0.00,0.00,0.00,0.00
1802,1001472.0,1001464.0,0.0,0.00,0.00,0.00,0.00,0.00
1803,1001472.0,1001466.0,0.0,0.00,0.00,0.00,0.00,0.00
1804,1001472.0,1001468.0,0.0,0.00,0.00,0.00,0.00,0.00


# Question 5: Calculate Time-Based Toll Rates
Create a function named calculate_time_based_toll_rates that takes the DataFrame created in Question 3 as input and calculates toll rates for different time intervals within a day.

The resulting DataFrame should have these five columns added to the input: start_day, start_time, end_day, and end_time.

start_day, end_day must be strings with day values (from Monday to Sunday in proper case)
start_time and end_time must be of type datetime.time() with the values from time range given below.
Modify the values of vehicle columns according to the following time ranges:

Weekdays (Monday - Friday):

From 00:00:00 to 10:00:00: Apply a discount factor of 0.8
From 10:00:00 to 18:00:00: Apply a discount factor of 1.2
From 18:00:00 to 23:59:59: Apply a discount factor of 0.8
Weekends (Saturday and Sunday):

Apply a constant discount factor of 0.7 for all times.
For each unique (id_start, id_end) pair, cover a full 24-hour period (from 12:00:00 AM to 11:59:59 PM) and span all 7 days of the week (from Monday to Sunday).

In [11]:
import pandas as pd
from datetime import time

def calculate_time_based_toll_rates(unrolled_df):
    #define time ranges and discount factors for weekdays and weekends
    weekday_time_ranges = [(time(0, 0, 0), time(10, 0, 0)),
                           (time(10, 0, 0), time(18, 0, 0)),
                           (time(18, 0, 0), time(23, 59, 59))]
    
    weekend_time_range = (time(0, 0, 0), time(23, 59, 59))

    weekday_discount_factors = [0.8, 1.2, 0.8]
    weekend_discount_factor = 0.7

    #create new columns for time based rates
    unrolled_df['start_day'] = ''
    unrolled_df['start_time'] = ''
    unrolled_df['end_day'] = ''
    unrolled_df['end_time'] = ''
    unrolled_df['time_based_rate'] = 0.0

    #iterate over each unique (id_start, id_end) pair
    for idx, group in unrolled_df.groupby(['id_start', 'id_end']):
        for day in range(7):  # Span all 7 days of the week
            for start_time, end_time in weekday_time_ranges:
                #apply weekday discount factors based on time ranges
                unrolled_df.loc[group.index, 'start_day'] = pd.to_datetime(group['start_day']).dt.day_name()
                unrolled_df.loc[group.index, 'start_time'] = start_time
                unrolled_df.loc[group.index, 'end_day'] = pd.to_datetime(group['start_day']).dt.day_name()
                unrolled_df.loc[group.index, 'end_time'] = end_time
                unrolled_df.loc[group.index, 'time_based_rate'] = weekday_discount_factors[day % len(weekday_discount_factors)]

            #apply constant weekend discount factor for all times
            unrolled_df.loc[group.index, 'start_day'] = pd.to_datetime(group['start_day']).dt.day_name()
            unrolled_df.loc[group.index, 'start_time'] = weekend_time_range[0]
            unrolled_df.loc[group.index, 'end_day'] = pd.to_datetime(group['start_day']).dt.day_name()
            unrolled_df.loc[group.index, 'end_time'] = weekend_time_range[1]
            unrolled_df.loc[group.index, 'time_based_rate'] = weekend_discount_factor

    #calqulate the final toll rates based on the time based rates
    for vehicle_type in ['moto', 'car', 'rv', 'bus', 'truck']:
        unrolled_df[vehicle_type] *= unrolled_df['time_based_rate']

    return unrolled_df


#we are assuming resulting_unrolled_df is the df from question 3
resulting_df_with_time_based_rates = calculate_time_based_toll_rates(resulting_unrolled_df)




In [12]:
resulting_df_with_time_based_rates

,id_start,id_end,distance,moto,car,rv,bus,truck,start_day,start_time,end_day,end_time,time_based_rate
0,1001400.0,1001402.0,9.7,5.432,8.148,10.185,14.938,24.444,NaN,00:00:00,NaN,23:59:59,0.7
1,1001400.0,1001404.0,0.0,0.000,0.000,0.000,0.000,0.000,NaN,00:00:00,NaN,23:59:59,0.7
2,1001400.0,1001406.0,0.0,0.000,0.000,0.000,0.000,0.000,NaN,00:00:00,NaN,23:59:59,0.7
3,1001400.0,1001408.0,0.0,0.000,0.000,0.000,0.000,0.000,NaN,00:00:00,NaN,23:59:59,0.7
4,1001400.0,1001410.0,0.0,0.000,0.000,0.000,0.000,0.000,NaN,00:00:00,NaN,23:59:59,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1801,1001472.0,1001462.0,0.0,0.000,0.000,0.000,0.000,0.000,NaN,00:00:00,NaN,23:59:59,0.7
1802,1001472.0,1001464.0,0.0,0.000,0.000,0.000,0.000,0.000,NaN,00:00:00,NaN,23:59:59,0.7
1803,1001472.0,1001466.0,0.0,0.000,0.000,0.000,0.000,0.000,NaN,00:00:00,NaN,23:59:59,0.7
1804,1001472.0,1001468.0,0.0,0.000,0.000,0.000,0.000,0.000,NaN,00:00:00,NaN,23:59:59,0.7
